In [ ]:
#Set up enviroment in google colab to run pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
#bring in Pyspark functions into your session

In [ ]:
from pyspark.sql.functions import *

In [ ]:
#bring data into your session

In [46]:
mydata = spark.read.format("csv").option("header","true").load("original.csv")

In [47]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
#check data types

In [48]:
mydata.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [49]:
mydata.head(5)
mydata.first()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [50]:
#view  your columns
mydata.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [ ]:
#Clean your data-  Remove nulls,   check for duplicates, remove duplicates

In [51]:
mydata2= mydata.withColumn("clean_city", when(mydata.City.isNull(),'Unknown').otherwise(mydata.City))
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [52]:
mydata2.count()

1000

In [53]:
#total number of unique rows
mydata2.distinct().count()

1000

In [54]:
#create new column new data type
mydata2= mydata2.withColumn("clean_salary", mydata2.Salary.substr(2,100).cast('float'))

In [ ]:
mydata2.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|  57438.18|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|   62846.6|     62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|  57576.52|    57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  61489.23|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6

In [56]:
mydata2_clean= mydata2.dropna()
mydata2_clean.count()

997

In [ ]:
mydata2_clean2=  mydata2.filter(mydata2.JobTitle.isNotNull())
mydata2_clean2.show()
#row 3 is now gone

In [58]:
#drop duplicates
mydata2_no_dups= mydata2.dropDuplicates()
mydata2_no_dups.show()

+---+-----------+----------+------+----------+--------------------+---------+-----------+-----------+----------+------------+
| id| first_name| last_name|gender|      City|            JobTitle|   Salary|   Latitude|  Longitude|clean_city|clean_salary|
+---+-----------+----------+------+----------+--------------------+---------+-----------+-----------+----------+------------+
|334|      Jared|  McCluney|  Male|    Garbów| Clinical Specialist|$27445.93|  51.370484| 22.3461706|    Garbów|    27445.93|
|591|   Mallorie|    Kairns|Female|     Skene|  Nurse Practicioner|$76849.72| 57.4924284| 12.6475194|     Skene|    76849.72|
|773|       Inna|    Worton|Female|   Wenquan|Compensation Analyst|$82283.41|  31.364042| 108.520914|   Wenquan|    82283.41|
|842|    Harbert|Clemmensen|  Male|      Puma|    Internal Auditor|$90772.00| -6.8316639| 39.2827032|      Puma|     90772.0|
|877|   Olivette|  Ghidelli|Female|  Virginia|Administrative Of...|$67823.45|  53.377533| -6.2633149|  Virginia|    67

In [59]:
mydata2.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+-------------------+-----------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|         Latitude|         Longitude|         clean_city|     clean_salary|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+-------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|              999|              1000|               1000|             1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null|25.43151724234234|43.337564614499996|               null|55487.95562890625|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|

In [60]:
#Grouping data - great for analysis
mean= mydata2.groupBy('JobTitle').avg('clean_salary')
mean.show(10)

+--------------------+------------------+
|            JobTitle| avg(clean_salary)|
+--------------------+------------------+
|Systems Administr...|  66131.4248046875|
|   Media Manager III|46968.356770833336|
|  Recruiting Manager|61231.947591145836|
|       Geologist III|    44579.80078125|
|        Geologist II|   43293.865234375|
|Database Administ...|     52018.4609375|
|   Financial Analyst|   62959.833203125|
|  Analyst Programmer|     37449.0921875|
|Software Engineer II|      74782.640625|
|       Accountant IV|   82732.248046875|
+--------------------+------------------+
only showing top 10 rows



In [61]:
#selecing  data
mydata2_select= mydata2.filter((mydata2.first_name== 'Roth'))
mydata2_select.show()

+---+----------+----------+------+------+--------------------+---------+---------+----------+----------+------------+
| id|first_name| last_name|gender|  City|            JobTitle|   Salary| Latitude| Longitude|clean_city|clean_salary|
+---+----------+----------+------+------+--------------------+---------+---------+----------+----------+------------+
|  9|      Roth|O'Cannavan|  Male|Heitan|VP Product Manage...|$73697.10|32.027934|106.657113|    Heitan|     73697.1|
+---+----------+----------+------+------+--------------------+---------+---------+----------+----------+------------+



In [62]:
mydata2_select= mydata2.select("first_name")
mydata2_select.show()

+----------+
|first_name|
+----------+
|   Melinde|
|  Kimberly|
|    Alvera|
|   Shannon|
|  Sherwood|
|     Maris|
|     Masha|
|   Goddart|
|      Roth|
|      Bran|
|    Kylynn|
|       Rey|
|      Kerr|
|    Mickie|
|    Kaspar|
|    Norbie|
|    Claude|
|     Thain|
|  Tiffanie|
|    Ettore|
+----------+
only showing top 20 rows



In [ ]:
#using SQL

In [63]:
mydata = spark.read.format("csv").option("header","true").load("original.csv")

In [64]:
from pyspark.sql.types import *

In [65]:
#use registerTempTable to create a tempuare table that I can use sql commands on
mydata.registerTempTable("mydata")

In [66]:
query1 = spark.sql('select * from mydata')
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [67]:
query2 = spark.sql('select * from mydata where first_name = "Roth"')
query2.show()


+---+----------+----------+------+------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|  City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+------+--------------------+---------+---------+----------+
|  9|      Roth|O'Cannavan|  Male|Heitan|VP Product Manage...|$73697.10|32.027934|106.657113|
+---+----------+----------+------+------+--------------------+---------+---------+----------+



In [ ]:
query2.write.csv('example.csv')